In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
project_id = ''

## Fetch data 

We start by fetching ~30K records from the original dataset:

In [ ]:
sql = """SELECT 
              unique_key
            , taxi_id
            , trip_start_timestamp
            , trip_end_timestamp
            , trip_seconds
            , trip_miles
            , pickup_census_tract
            , dropoff_census_tract
            , pickup_community_area
            , dropoff_community_area
            , pickup_latitude
            , pickup_longitude
            , dropoff_latitude
            , dropoff_longitude
            , payment_type
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
WHERE RAND() < (30000/112860054) 
AND  trip_start_timestamp < '2016-01-01' """
df = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [ ]:
# Plotting missing values rate
(df.isnull().sum()/(30000)).plot(kind='bar', figsize=(10,3))
plt.title('Precentage of missing values by feature')
plt.show()

Our first obsercation that many pickup and dropoff locations are missing, <br> 
since our goal is to predict trip duraion we have to know the origin and destination of the trip. <br>
For this reason, when building the dataset we will filter missing destination and origin rows

In [ ]:
df = df.dropna()

In [ ]:
df[['trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds']].head()

We note that the trip start and end time are rounded to 15 minutes <br>
However trip_seconds is not round, so it probably reflects a more accurate measurement of the ride time

## Analyzing features distributions

In [ ]:
df['trip_start_timestamp'] = df['trip_start_timestamp'].apply(pd.Timestamp)
df['trip_end_timestamp'] = df['trip_end_timestamp'].apply(pd.Timestamp)

Trip trends over the years:

In [ ]:
total_trips = df.set_index('trip_start_timestamp').sort_index()['trip_miles'].fillna(0).rolling('7d').count()
total_trips.plot(lw=0.8, figsize=(20,7))
plt.ylim(20,)
plt.title('Trip count per week', size=20)
plt.ylabel('Trip count in the dataset', size=20)
plt.grid()
plt.show()

In [ ]:
df['trip_speed_kmph'] = (df['trip_miles'] * (1.60934)) / ((df['trip_seconds'] + .01)/ (60 * 60))

trip_speed = df.set_index('trip_start_timestamp').sort_index()['trip_speed_kmph'].fillna(0).rolling('7d').median()
trip_speed.plot(lw=0.8, figsize=(20,7))
plt.ylim(3,30)
plt.title('Weekly Median Trip Speed', size=20)
plt.ylabel('Median trip speed (kmph)', size=20)
plt.grid()
plt.show()

There doesn't seem to be a clear trend to the total number of rides or to the travel speed

## Filtering out unreasonable trip speeds

In [ ]:
# Impute data
df = df[df['trip_miles'] > 0.2] # Filter very short trips
df = df[df['trip_seconds'] > 2] # Filter very short trips
df = df[df['trip_speed_kmph'].between(5,120)] # Filter very FAST and very SLOW trips

In [ ]:
df['trip_speed_kmph'].dropna().plot(kind='hist', bins=range(0,90, 2), alpha=0.5, density=True)
plt.show()

In [ ]:
total_trips = df.set_index('trip_start_timestamp').sort_index()['trip_speed_kmph'].fillna(0).rolling('30d', min_periods=30).mean()
total_trips.plot(lw=0.8, figsize=(20,7))
plt.grid()
plt.show()

## Exemine the effect of the trip date and time on the trip duration

In [ ]:
df['hour_of_day'] = df['trip_start_timestamp'].dt.hour
df['day_of_week'] = df['trip_start_timestamp'].dt.weekday

In [ ]:
fig = plt.figure(figsize=(20,6))
plt.subplot(1, 2, 1)
sns.boxplot(x='hour_of_day', y='trip_speed_kmph', data=df, showfliers=False)
plt.subplot(1, 2, 2)
sns.boxplot(x='day_of_week', y='trip_speed_kmph', data=df, showfliers=False)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,6))
plt.subplot(1, 2, 1)
sns.boxplot(x='hour_of_day', y='trip_seconds', data=df, showfliers=False)
plt.subplot(1, 2, 2)
sns.boxplot(x='day_of_week', y='trip_seconds', data=df, showfliers=False)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.pivot_table(index='pickup_community_area', columns='dropoff_community_area',
                           values='trip_seconds'), cmap='Blues')
plt.show()

## Training a model 

try to train a simple LightGBM model using the sklearn API and measure the mean absolute error

In [ ]:
#  https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/sklearn_example.py